<a href="https://colab.research.google.com/github/cmari038/Language-Translator/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install spacy
#!pip install collections

In [2]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 43.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import torch
import torchtext; torchtext.disable_torchtext_deprecation_warning()
import pandas as pd
import numpy as np
import spacy
import torch.nn as nn
import torch.nn.functional as functional
from torchtext.data.utils import get_tokenizer
from torch.utils.data import random_split
from torchtext.vocab import vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter, OrderedDict

In [62]:
# connect pytorch to cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Cuda activated")
else:
    device = torch.device("cpu")

dataset = 'https://raw.githubusercontent.com/cmari038/Language-Translator/main/data.csv'
data = pd.read_csv(dataset)

#spacy.load('en_core_web_sm')
#spacy.load('es_core_news_sm')

# processing data

english_tokenizer = get_tokenizer('spacy', language = 'en_core_web_sm')
spanish_tokenizer = get_tokenizer('spacy', language= 'es_core_news_sm')

#print(data)

#train = data.sample(frac=0.7)
#validate = data.drop(train.index).sample(frac=0.1)
#test = data.drop(validate.index)

train = data.iloc[:int(len(data) * 0.75)]

en_lowercase = []
for element in train['english']:
  en_lowercase.append(element.lower())

train['english'] = en_lowercase

es_lowercase = []
for element in train['spanish']:
  es_lowercase.append(element.lower())

train['spanish'] = es_lowercase

#train.reset_index(drop=True, inplace=True)
#validate.reset_index(drop=True, inplace=True)

# Build vocab structure to store tokens and their corresponding index
counter1 = Counter()
counter2 = Counter()

for sentence in train['english']:
  counter1.update(english_tokenizer(sentence))

for sentence in train['spanish']:
  counter2.update(spanish_tokenizer(sentence))

en_dict = OrderedDict(counter1.most_common())
es_dict = OrderedDict(counter2.most_common())

vocab1 = vocab(en_dict, specials = ['<unk>', '<pad>', '<sos>', '<eos>'])
vocab2 = vocab(es_dict, specials = ['<unk>', '<pad>', '<sos>', '<eos>'])

vocab1.set_default_index(vocab1['<unk>'])
vocab2.set_default_index(vocab2['<unk>'])

Cuda activated


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
<ipython-input-62-2ee575e0010d>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['english'] = en_lowercase
<ipython-input-62-2ee575e0010d>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [63]:
print(train)

                                      english  \
0                                         go.   
1                                         go.   
2                                         go.   
3                                         go.   
4                                         hi.   
...                                       ...   
89218  tom acknowledges that he was defeated.   
89219  tom admitted to spilling the red wine.   
89220  tom adopted our method of bookkeeping.   
89221  tom agreed to help mary with her work.   
89222  tom almost always goes to work by car.   

                                                 spanish  
0                                                    ve.  
1                                                  vete.  
2                                                  vaya.  
3                                                váyase.  
4                                                  hola.  
...                                                  ... 

In [66]:
# finding set of tokens for each phrase
def getTokens(df, en_tokenizer, es_tokenizer, lang):
  if lang == "english":
    en_tokens = []

    for token in en_tokenizer(df):
      en_tokens.append(token)

    en_tokens = ['<sos>'] + en_tokens + ['<eos>']

    return en_tokens

  else:
    es_tokens = []

    for token in es_tokenizer(df):
        es_tokens.append(token)

    es_tokens = ['sos'] + es_tokens + ['<eos>']

    return es_tokens


#token_dict = {"en_tokenizer": english_tokenizer, "es_tokenizer": spanish_tokenizer}
#train = train.apply(map(lambda col: col.map(getTokens)))
en_tokens = []
es_tokens = []

for element in train['english']:
  tmp1 = getTokens(element, english_tokenizer, spanish_tokenizer, "english")
  en_tokens.append(tmp1)

for element in train["spanish"]:
  tmp2 = getTokens(element, english_tokenizer, spanish_tokenizer, "spanish")
  es_tokens.append(tmp2)

train['en_tokens'] = en_tokens
train['es_tokens'] = es_tokens

#print(train['en_tokens'])

<ipython-input-66-d2fb3b41a86f>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['en_tokens'] = en_tokens
<ipython-input-66-d2fb3b41a86f>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['es_tokens'] = es_tokens


In [67]:
print(train['en_tokens'])

0                                    [<sos>, go, ., <eos>]
1                                    [<sos>, go, ., <eos>]
2                                    [<sos>, go, ., <eos>]
3                                    [<sos>, go, ., <eos>]
4                                    [<sos>, hi, ., <eos>]
                               ...                        
89218    [<sos>, tom, acknowledges, that, he, was, defe...
89219    [<sos>, tom, admitted, to, spilling, the, red,...
89220    [<sos>, tom, adopted, our, method, of, bookkee...
89221    [<sos>, tom, agreed, to, help, mary, with, her...
89222    [<sos>, tom, almost, always, goes, to, work, b...
Name: en_tokens, Length: 89223, dtype: object


In [68]:
def getIndices(df, en_vocab, es_vocab, lang):
  if lang == "english":
    en_indices = []

    for word in df:
      en_indices.append(en_vocab[word])

    return en_indices

  else:
    es_indices = []

    for word in df:
        es_indices.append(es_vocab[word])

    return es_indices

en_indices = []
es_indices = []

for element in train["en_tokens"]:
  tmp1 = getIndices(element, vocab1, vocab2, "english")
  en_indices.append(tmp1)

for element in train["es_tokens"]:
  tmp1 = getIndices(element, vocab1, vocab2, "spanish")
  es_indices.append(tmp1)

train['es_indices'] = es_indices
train['en_indices']= en_indices

"""
print(train['en_indices'])

for series in train['en_indices']:
  en_tensor.append(torch.tensor(series, dtype=torch.long))

for series in train['es_indices']:
  es_tensor.append(torch.tensor(series, dtype=torch.long))

train['en_indices'] = en_tensor
train['es_indices'] = es_tensor
"""

<ipython-input-68-895ec81c67b4>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['es_indices'] = es_indices
<ipython-input-68-895ec81c67b4>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['en_indices']= en_indices


"\nprint(train['en_indices'])\n\nfor series in train['en_indices']:\n  en_tensor.append(torch.tensor(series, dtype=torch.long))\n\nfor series in train['es_indices']:\n  es_tensor.append(torch.tensor(series, dtype=torch.long))\n\ntrain['en_indices'] = en_tensor\ntrain['es_indices'] = es_tensor\n"

In [69]:
print(train['en_indices'])

0                                      [2, 48, 4, 3]
1                                      [2, 48, 4, 3]
2                                      [2, 48, 4, 3]
3                                      [2, 48, 4, 3]
4                                    [2, 1711, 4, 3]
                            ...                     
89218          [2, 8, 10160, 19, 15, 21, 3277, 4, 3]
89219      [2, 8, 1383, 7, 10161, 9, 404, 471, 4, 3]
89220        [2, 8, 1725, 143, 3185, 26, 5806, 4, 3]
89221      [2, 8, 993, 7, 87, 35, 45, 55, 103, 4, 3]
89222    [2, 8, 271, 147, 433, 7, 103, 91, 97, 4, 3]
Name: en_indices, Length: 89223, dtype: object


In [70]:
# convets dataframe to dataset for tensor acess
class TensorSet(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    return torch.tensor(self.df['en_indices'].iloc[index], dtype=torch.long), torch.tensor(self.df['es_indices'].iloc[index], dtype=torch.long)

def collate_fn(batch):
  # used for making sure sequences are similar lengths by adding tokens to pad out the length
  en_batch = []
  es_batch = []
  for en_sample, es_sample in batch:
    en_batch.append(en_sample)
    es_batch.append(es_sample)

  en_batch = pad_sequence(en_batch, padding_value=vocab1['<pad>'])
  es_batch = pad_sequence(es_batch, padding_value=vocab2['<pad>'])

  return en_batch, es_batch

#tensorSet = TensorSet(train, english_tokenizer, spanish_tokenizer, vocab1, vocab2)
tensorSet = TensorSet(train)
batch_size = 128
dataLoad = DataLoader(tensorSet, shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
#evalDataLoad = DataLoader(TensorSet(validate), shuffle=True, collate_fn=collate_fn, batch_size=batch_size)

In [71]:
# RNN model
# made up of encoder, deocder, and seq2seq
# encoder converts tensors into context vectors
class RNN_Encoder(nn.Module):
    def __init__(self, input, embedding_dimension, hidden_dimension, layers, dropout_p=0.5):
        super(RNN_Encoder, self).__init__()
        self.hidden_dimension = hidden_dimension
        self.layers = layers
        self.embed = nn.Embedding(input, embedding_dimension)
        self.gru = nn.LSTM(embedding_dimension, hidden_dimension, layers, dropout=dropout_p)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, english):
        embedded = self.dropout(self.embed(english))
        output, (hidden, cell) = self.gru(embedded)
        return hidden, cell

# decoder uses context vectors to output spanish translation
class RNN_Decoder(nn.Module):
    def __init__(self, output, embedding_dimension, hidden_dimension, layers, dropout_p=0.5):
      super(RNN_Decoder, self).__init__()
      self.output = output
      self.hidden_dimension = hidden_dimension
      self.layers = layers
      self.embed = nn.Embedding(output, hidden_dimension)
      self.gru = nn.LSTM(embedding_dimension, hidden_dimension, layers, dropout=dropout_p)
      self.fc_out = nn.Linear(hidden_dimension, output)
      self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden, cell):
      input = input.unsqueeze(0)
      embedded = self.dropout(self.embed(input))
      out, (hidden, cell) = self.gru(embedded, (hidden, cell))
      prediction = self.fc_out(out.squeeze(0))
      #output = self.embed(input)
      #output = functional.relu(output)
      #output, hidden = self.gru(output, hidden)
      return prediction, hidden, cell

# sequence to sequence that ties encoder and decoder together
class Sequence(nn.Module):
    def __init__(self, encoder, decoder, device):
      super(Sequence, self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.device = device

    def forward(self, english, spanish, teacher_forcing_ratio):
      batch_size = spanish.shape[1]
      es_length = spanish.shape[0]
      es_vocab_size = self.decoder.output
      outputs = torch.zeros(es_length, batch_size,es_vocab_size).to(self.device)
      hidden, cell = self.encoder(english)
      input = spanish[0,:]

      for i in range(1, es_length):
        output, hidden, cell = self.decoder(input, hidden, cell)
        outputs[i] = output
        teacher_force = np.random.random() < teacher_forcing_ratio
        top1 = output.argmax(1)
        decoder_input = spanish[i] if teacher_force else top1

      return outputs


encoder = RNN_Encoder(input=len(vocab1), embedding_dimension=256, hidden_dimension=256, layers=2)
decoder = RNN_Decoder(output=len(vocab2), embedding_dimension=256, hidden_dimension=256, layers=2)
RNN_model = Sequence(encoder, decoder, device).to(device)

In [72]:
criterion = nn.CrossEntropyLoss(ignore_index=vocab2['<pad>'])
optimizer = torch.optim.Adam(RNN_model.parameters())

def weights(model):
  for name, parameter in model.named_parameters():
    nn.init.uniform_(parameter.data, -0.8, 0.8)

def training(RNN_model, dataLoad, criterion, optimizer, device, epochs, teacher_forcing_ratio, clip):
  #RNN_model.train()
  for epoch in range(epochs):
    RNN_model.train()
    epoch_loss = 0
    for i, (english, spanish) in enumerate(dataLoad): # iterate in batches through dataloader
      english = english.to(device)
      spanish = spanish.to(device)
      optimizer.zero_grad()

      output = RNN_model(english, spanish, teacher_forcing_ratio)
      output = output[1:].view(-1, output.shape[-1])
      spanish = spanish[1:].view(-1)

      loss = criterion(output, spanish)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(RNN_model.parameters(),clip)
      optimizer.step()
      epoch_loss += loss.item()

RNN_model.apply(weights)
training(RNN_model, dataLoad, criterion, optimizer, device, 5, 0.5, 1.0)

In [78]:
# translates an input sentece into spanish using trained model
def translation(model, input, vocab1, vocab2, device):
  input = input.lower()
  model.eval()
  #tokens = english_tokenizer(input)
  #indices = []
  """for token in tokens:
      indices.append(vocab1[token])
  input_tensor = torch.tensor([vocab1['<sos>']] + indices + [vocab1['<eos>']], dtype=torch.long) """

  with torch.no_grad():
    tokens = getTokens(input, english_tokenizer, spanish_tokenizer, "english")
    indices = getIndices(tokens, vocab1, vocab2, "english")
    #print(indices)
    tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(-1).to(device)
    #print(tensor)
    hidden, cell = model.encoder(tensor)
    spanish = [vocab2['<sos>']]

    """encoder_output, hidden = model.encoder(tensor, len(tokens))
    spanish_vocab = {i: word for word, i in spanish_vocab.items()}
    decoder_input = torch.tensor([vocab2['<sos>']]).to(device)
    spanish = [] """

    for i in range(50):
        tensorInput = torch.tensor([spanish[-1]], dtype=torch.long).to(device)
        decoder_output, hidden, cell = model.decoder(tensorInput, hidden, cell)
        top1 = decoder_output.argmax(-1).item()
        if top1 == vocab2['<eos>']:
          spanish.append(top1)
          break
        spanish.append(top1)
        decoder_input = torch.tensor([top1]).to(device)
    tokens = vocab2.lookup_tokens(spanish)
  return tokens


#train(RNN_model, dataLoad, criterion, optimizer, device, 10, 0.5)
sentence = "The dog is running towards me"
translated_sentence = translation(RNN_model, sentence, vocab1, vocab2, device)
print(translated_sentence)

['<sos>', 'él', 'tenemos', 'éxitos', 'ellos', 'me', '.', ',', 'que', '.', '<eos>']
